# Quantum Software Development Journey: 
# From Theory to Application with Classiq

## Final Assignment - Overview

In this homework assignment, you have the opportunity to implement something meaningful, be creative, and practice your new skills and knowledge! You can apply your knowledge from the course and contribute to the Classiq community. Details are provided below.

**Successful contributions will have the opportunity to earn SWAG credits and potentially more advanced certificates.**

## The Project

### Goal
The main goal is to be creative, collaborate with each other and with the community, and to apply your knowledge to bridge the gap between theory and application! Focus on the last two weeks of the course, using this assignment to solidify and expand your knowledge and skills.

### Optional Projects
You may choose a project from the following list, or come up with your own idea. Just please make sure to consult with us before you start!

- **Domain of Expertise**:
  - Use your background in neural networks, chemistry, biology, or any other field to implement something new. We are here to support you!
  - Example: An advanced application of VQE for a more complex molecule.
  - Implementation of different QNN or QAE architectures.
  - Combine your domain of expertise with the power of Classiq!

- **QNN for XOR Problem**:
  - Classiq has an available dataset for training PQC to imitate the XOR gate, similar to how we trained a U-gate to act as a NOT gate. Design a QNN to solve the XOR problem. Read more on the dataset [here](https://docs.classiq.io/latest/reference-manual/built-in-algorithms/qml/qnn/datasets/#datasetxor).

- **QNN as VQE**:
  - Like VQE, QNNs with well-suited loss functions and data can be used to find minima of a system. Can you solve one of the problems from week 3 (in class or HW3) by implementing it with QNN?
  - Review the materials from weeks 3 and 4 and tackle this problem!
  - If needed, you may create synthetic data.
  - Do you have another example to show how QNN can generalize VQE? Show us!

- **Noise Reduction Using Quantum Auto-Encoders**:
  - Quantum Auto-Encoders can be used to reduce noise!
  - Create code that generates a quantum state, adds random noise to it, and tries to reconstruct it using a Quantum Auto-Encoder.
  - If needed, you may create synthetic data.

- **Contribute to Quantum Algorithm Zoo**:
  - Implement one of the algorithms in the [Quantum Algorithm Zoo](https://quantumalgorithmzoo.org/) that has not been implemented yet using Classiq.
  - By doing so, you will have the opportunity to contribute to one of the main resources on quantum algorithms! (your Implementation will be linked to their website!)

- **New Algorithm Implementation**:
  - Choose a research paper (you may consult us) and try to implement it using Classiq's SDK.

### Note

- For those who choose a more extensive project, you will have a discussion with me or another Classiq member to fine-tune your project's purpose and set a deadline. This collaborative approach ensures your project aligns with course objectives and maximizes your learning experience.
- **You are allowed to work in teams of up to 3 members!**

### Deadline & Submission

- **Important Dates**:
  - **Assignment Release:** 29.5.2024
  - **Submission Deadline:** 10.6.2024 (7 A.M GMT+3)
- Consult with us before submitting your project, and we will direct you to the right place in the [Classiq Library](https://github.com/Classiq/classiq-library).
- You might get an extension to the deadline based on your specific project and progress.

## Conclusion

Choose the project that best aligns with your interests and career goals. This project provides a valuable opportunity to deepen your understanding of QML and quantum computing in general, while contributing to the Classiq community.

If you have any questions or need further clarification, feel free to reach out.

**Happy coding!**


In [1]:
from classiq import (
    CX,
    RY,
    CArray,
    CInt,
    CReal,
    Input,
    Output,
    QArray,
    QBit,
    allocate,
    bind,
    create_model,
    qfunc,
    repeat,
)
from classiq.qmod.symbolic import pi

In [ ]:
@qfunc
def angle_encoding(exe_params: CArray[CReal], qbv: Output[QArray[QBit]]) -> None:
    allocate(exe_params.len, qbv)
    repeat(
        count=exe_params.len,
        iteration=lambda index: RY(pi * exe_params[index], qbv[index]),
    )

In [ ]:
@qfunc
def encoder_ansatz(
    num_qubits: CInt,
    num_encoding_qubits: CInt,
    exe_params: CArray[CReal],
    x: Input[QArray[QBit, "num_qubits"]],
    trash: Output[QArray[QBit, "num_qubits-num_encoding_qubits"]],
    coded: Output[QArray[QBit, "num_encoding_qubits"]],
) -> None:
    """
    This is a parametric model which has num_trash_qubits = num_qubits-num_encoding_qubits as an output.
    It contains num_trash_qubits layers, each composed of RY gates and CX gates with a linear connectivity,
    and a final layer with RY gate on each of the trash qubits is applied.
    """

    def single_layer(rep: CInt) -> None:
        repeat(
            count=num_qubits,
            iteration=lambda index: RY(exe_params[rep * num_qubits + index], x[index]),
        )
        repeat(
            count=num_qubits - 1,
            iteration=lambda index: CX(x[index], x[index + 1]),
        )

    repeat(count=num_qubits - num_encoding_qubits, iteration=single_layer)
    bind(x, [coded, trash])
    repeat(
        count=num_qubits - num_encoding_qubits,
        iteration=lambda index: RY(
            exe_params[(num_qubits - num_encoding_qubits) * num_qubits + index],
            trash[index],
        ),
    )

In [ ]:
import numpy as np

domain_wall_data = np.array([[0, 0, 1, 1], [0, 0, 0, 1], [0, 1, 1, 1]])
print("domain wall data:\n", domain_wall_data)

In [ ]:
from classiq import show, swap_test, synthesize

NUM_QUBITS = 4
NUM_ENCODING_QUBITS = 2
num_trash_qubits = NUM_QUBITS - NUM_ENCODING_QUBITS
num_weights_in_encoder = NUM_QUBITS * num_trash_qubits + num_trash_qubits

In [ ]:
@qfunc
def main(
    w: CArray[CReal, num_weights_in_encoder],
    input: CArray[CReal, NUM_QUBITS],
    trash: Output[QArray[QBit, num_trash_qubits]],
    coded: Output[QArray[QBit, NUM_ENCODING_QUBITS]],
    test: Output[QBit],
) -> None:
    x = QArray("x")
    psi2 = QArray("psi2")
    allocate(num_trash_qubits, psi2)
    angle_encoding(exe_params=input, qbv=x)
    encoder_ansatz(
        num_qubits=NUM_QUBITS,
        num_encoding_qubits=NUM_ENCODING_QUBITS,
        exe_params=w,
        x=x,
        trash=trash,
        coded=coded,
    )

    swap_test(state1=trash, state2=psi2, test=test)


ae_qmod = create_model(main)

In [ ]:
qprog = synthesize(ae_qmod)
show(qprog)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from classiq.applications.qnn import QLayer
from classiq.applications.qnn.types import (
    MultipleArguments,
    ResultsCollection,
    SavedResult,
)
from classiq.execution import (
    ExecutionPreferences,
    execute_qnn,
    set_quantum_program_execution_preferences,
)
from classiq.synthesis import SerializedQuantumProgram

In [ ]:
num_shots = 4096


def execute(
    quantum_program: SerializedQuantumProgram, arguments: MultipleArguments
) -> ResultsCollection:
    quantum_program = set_quantum_program_execution_preferences(
        quantum_program, preferences=ExecutionPreferences(num_shots=num_shots)
    )
    return execute_qnn(quantum_program, arguments)


def post_process(result: SavedResult) -> torch.Tensor:
    alpha_sqaured = result.value.counts_of_output("test")["0"] / num_shots
    out = 1 - alpha_sqaured
    return torch.tensor(out)

In [ ]:
def create_net(*args, **kwargs) -> nn.Module:
    class Net(nn.Module):
        def __init__(self, *args, **kwargs):
            super().__init__()

            self.qlayer = QLayer(
                qprog,
                execute,
                post_process,
                *args,
                **kwargs,
            )

        def forward(self, x):
            x = self.qlayer(x)
            return x

    return Net(*args, **kwargs)


encoder_train_network = create_net()

In [ ]:
class MyDWDataset:
    def __init__(self, data, labels) -> None:
        self.data = torch.from_numpy(data).float()
        self.labels = torch.unsqueeze(torch.from_numpy(labels), dim=-1).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
labels = np.array([0, 0, 0])
train_dataset = MyDWDataset(domain_wall_data, labels)
train_data_loader = DataLoader(
    train_dataset, batch_size=2, shuffle=True, drop_last=False
)

In [ ]:
import time as time


def train(
    model: nn.Module,
    data_loader: DataLoader,
    loss_func: nn.modules.loss._Loss,
    optimizer: optim.Optimizer,
    epoch: int = 40,
) -> None:
    for index in range(epoch):
        start = time.time()
        for data, label in data_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = loss_func(torch.squeeze(output), torch.squeeze(label))
            loss.backward()
            optimizer.step()

        print(time.time() - start)
        print(index, f"\tloss = {loss.item()}")

In [ ]:
_LEARNING_RATE = 0.3
loss_func = nn.L1Loss()
optimizer = optim.SGD(encoder_train_network.parameters(), lr=_LEARNING_RATE)

In [ ]:
trained_weights = torch.nn.Parameter(
    torch.Tensor(
        [1.5227, 0.3588, 0.6905, 1.4777, 1.5718, 1.5615, 1.5414, 0.6021, 0.1254, 0.9903]
    )
)
encoder_train_network.qlayer.weight = trained_weights

In [ ]:
data_loader = train_data_loader

train(encoder_train_network, data_loader, loss_func, optimizer, epoch=1)

In [ ]:
@qfunc
def encoder_ansatz_wrapper(
    num_qubits: CInt,
    num_encoding_qubits: CInt,
    exe_params: CArray[CReal],
    qbv: QArray[QBit, "num_qubits"],
) -> None:
    coded = QArray("coded")
    trash = QArray("trash")
    encoder_ansatz(
        num_qubits=num_qubits,
        num_encoding_qubits=num_encoding_qubits,
        exe_params=exe_params,
        x=qbv,
        trash=trash,
        coded=coded,
    )
    bind([coded, trash], qbv)

In [ ]:
from classiq import invert


@qfunc
def main(
    w: CArray[CReal, num_weights_in_encoder],
    input: CArray[CReal, NUM_QUBITS],
    decoded: Output[QArray[QBit, NUM_QUBITS]],
    trash: Output[QArray[QBit, num_trash_qubits]],
) -> None:
    psi2 = QArray("psi2")
    coded = QArray("coded")
    allocate(num_trash_qubits, psi2)
    angle_encoding(exe_params=input, qbv=decoded)
    encoder_ansatz(
        num_qubits=NUM_QUBITS,
        num_encoding_qubits=NUM_ENCODING_QUBITS,
        exe_params=w,
        x=decoded,
        trash=trash,
        coded=coded,
    )

    bind([coded, psi2], decoded)
    invert(
        operand=lambda: encoder_ansatz_wrapper(
            num_qubits=NUM_QUBITS,
            num_encoding_qubits=NUM_ENCODING_QUBITS,
            exe_params=w,
            qbv=decoded,
        ),
    )


validator_qmod = create_model(main)

In [ ]:
validator_qprog = synthesize(validator_qmod)
show(validator_qprog)

In [ ]:
def execute_validator(
    quantum_program: SerializedQuantumProgram, arguments: MultipleArguments
) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

In [ ]:
def post_process_validator(result: SavedResult) -> torch.Tensor:
    counts = result.value.counts_of_output("decoded")

    max_key = max(counts, key=counts.get)

    return torch.tensor([int(k) for k in max_key])

In [ ]:
def create_encoder_decoder_net(*args, **kwargs) -> nn.Module:
    class Net(nn.Module):
        def __init__(self, *args, **kwargs):
            super().__init__()

            self.qlayer = QLayer(
                validator_qprog,
                execute_validator,
                post_process_validator,
                *args,
                **kwargs,
            )

        def forward(self, x):
            x = self.qlayer(x)
            return x

    return Net(*args, **kwargs)


validator_network = create_encoder_decoder_net()
validator_network.qlayer.weight = encoder_train_network.qlayer.weight

In [ ]:
validator_data_loader = DataLoader(
    train_dataset, batch_size=1, shuffle=True, drop_last=False
)

for data, label in validator_data_loader:
    output = validator_network(data)
    print("input=", data.tolist()[0], ",   output=", output.tolist()[0])

In [ ]:
input_anomaly_data = np.array(
    [[0, 0, 1, 1], [0, 0, 0, 1], [0, 1, 1, 1], [1, 0, 1, 0], [1, 1, 1, 1]]
)
anomaly_labels = np.array([0, 0, 0, 0, 0])
anomaly_dataset = MyDWDataset(input_anomaly_data, anomaly_labels)
anomaly_data_loader = DataLoader(
    anomaly_dataset, batch_size=1, shuffle=True, drop_last=False
)

In [ ]:
tolerance = 1e-2
for data, label in anomaly_data_loader:
    output = encoder_train_network(data)
    if abs(output.tolist()[0]) > tolerance:
        print("anomaly:", data.tolist()[0])

In [ ]:
@qfunc
def main(
    w: CArray[CReal, num_weights_in_encoder],
    input: CArray[CReal, NUM_QUBITS],
    trash: Output[QArray[QBit, num_trash_qubits]],
) -> None:
    x = QArray("x")
    coded = QArray("coded")
    angle_encoding(exe_params=input, qbv=x)
    encoder_ansatz(
        num_qubits=NUM_QUBITS,
        num_encoding_qubits=NUM_ENCODING_QUBITS,
        exe_params=w,
        x=x,
        trash=trash,
        coded=coded,
    )


ae_qmod = create_model(main)


qprog = synthesize(ae_qmod)
show(qprog)

In [ ]:
from classiq.applications.chemistry import PauliOperator, PauliOperators


def execute(
    quantum_program: SerializedQuantumProgram, arguments: MultipleArguments
) -> ResultsCollection:
    return execute_qnn(
        quantum_program,
        arguments,
        observable=PauliOperator(pauli_list=[("IZ", 1), ("ZI", 1)]),
    )


def post_process(result: SavedResult) -> torch.Tensor:
    out = 1 / 2 * (2 - np.real(result.value.value))
    return torch.tensor(out)

In [ ]:
def create_net(*args, **kwargs) -> nn.Module:
    class Net(nn.Module):
        def __init__(self, *args, **kwargs):
            super().__init__()

            self.qlayer = QLayer(
                qprog,
                execute,
                post_process,
                *args,
                **kwargs,
            )

        def forward(self, x):
            x = self.qlayer(x)
            return x

    return Net(*args, **kwargs)


encoder_train_network = create_net()